- declarations
- declare tables dict
- declare relations dict
- declare enums dict
- set target model
- parse line by line to tables/relations/enums
- flip to df
- hash it
- DB it

In [2]:
# Imports from dbdiagram.io format to logical model
# Requires:
#   TargetFilename : the name of the file to bring in
#   TargetModelKeyPhrase : the KeyPhrase for the model to import to

import sqlite3
import pandas as pd
import hashing
import re

conn = sqlite3.connect('app_db.sqlite')
cur = conn.cursor()

TargetPath = 'imports/logical/'
TargetFilename = 'Enterprise_KYC_dbdiagramio.txt'
TargetModelKeyPhrase = 'jam'
RecordSource = 'dbdiagramio'

TableList = []
TableAttributeList = []
RelationList = []
EnumList = []
EnumValueList = []

In [3]:
# splits the entity name out of a line
# ignores the schema name, as this is logical import
def ParseEntityName(line):
  LineList = line.split(' ')
  NameList = LineList[1].split('.')
  EntityName = NameList[1]
  return EntityName

In [4]:
# Parse the file to the lists
# currently doesn't do separate relations
# currently doesn't check for 'ive seen this entity before'. DV pattern will error on two things.

CurrentStructName = ''
CurrentStructType = '' # ENUM or TABLE
CurrentStructList = []

with open(TargetPath + TargetFilename) as file:
  for line in file:
    line = line.rstrip()
    #print(line)
    
    if line.startswith('//'):
      #print('ignore line, comment')
      pass
    elif line.startswith('Table '):
      #print('set new target table')
      CurrentStructType = 'TABLE'
      CurrentStructName = ParseEntityName(line)
      TableList.append([TargetModelKeyPhrase, CurrentStructName])
    elif line.startswith('  '):
      if CurrentStructType == 'TABLE':
        #print('read attribute to current TABLE')
        # start with the basics
        LineList = line.lstrip().split(' ')
        AttributeName = LineList[0]
        AttributeType = LineList[1]
        EnumName = ''
        if AttributeType.startswith('enum.'):
          AttributeList = AttributeType.split('.')
          AttributeType = AttributeList[0]
          EnumName = AttributeList[1]

        LineAdditional = re.findall("\[(.*)\]", line)
        if len(LineAdditional) > 0:
          #print(LineAdditional[0])
          AdditionalList = re.findall("(pk|(?:note|ref): ?(?=[^']|(')?)'?(?:(?(2)(?:[^'])*|[^,']*))'?)", LineAdditional[0])
          #print(AdditionalList)
          for AdditionalItem in AdditionalList:
            AdditionalItem = AdditionalItem[0] # the regex returns 2 matches, to do with the forward check
            if AdditionalItem == 'pk':
              isPK = True
            else:
              m = re.match("(?P<key>\w+): ?(?P<value>.*)", AdditionalItem)
              #print(m.groupdict())
              if m.groupdict()['key'] == 'ref':
                #print('inline relation description')
                RelationInfoList = m.groupdict()['value'].split(' ')
                
                RelationType = RelationInfoList[0];
                if RelationType == '>':
                  RelationType = 'M21'
                elif RelationType == '<':
                  RelationType = '12M'
                elif RelationType == '-':
                  RelationType = '121'
                elif RelationType == '<>':
                  RelationType = 'M2M'
                #print(RelationType)
                
                RelationTargetString = RelationInfoList[1];
                RelationTargetList = RelationTargetString.split('.')
                #RelationTargetSchema = RelationTargetList[0] # ignore schema, this is logical
                RelationTargetTable = RelationTargetList[1]
                RelationTargetAttribute = RelationTargetList[2]
                #print(RelationTargetTable)
                #print(RelationTargetAttribute)

                RelationList.append([
                  TargetModelKeyPhrase
                  , CurrentStructName
                  , AttributeName
                  , RelationTargetTable
                  , RelationTargetAttribute
                  , RelationType
                ])
                RelationTargetTable = ''
                RelationTargetAttribute = ''
                RelationType = ''

              elif m.groupdict()['key'] == 'note':
                AttributeNote = m.groupdict()['value'].strip('\'')
                #print(AttributeNote)
        
        TableAttributeList.append([
          TargetModelKeyPhrase
          , CurrentStructName
          , AttributeName
          , AttributeType
          , EnumName
          , isPK
          , AttributeNote
        ])
        AttributeName = ''
        AttributeType = ''
        EnumName = ''
        isPK = False
        AttributeNote = ''

      elif CurrentStructType == 'ENUM':
        #print('read EnumValue to current ENUM')
        EnumValue = line.lstrip().strip('"')
        EnumValueList.append([
          TargetModelKeyPhrase
          , CurrentStructName
          , EnumValue
        ])
        EnumValue = ''

    elif line.startswith('Enum '):
      #print('set new target enum')
      CurrentStructType = 'ENUM'
      CurrentStructName = ParseEntityName(line)
      EnumList.append([TargetModelKeyPhrase, CurrentStructName])
    elif line.startswith('}'):
      #print('close current target table or enum')
      CurrentStructName = ''
    else:
      pass

print(TableList)
print(TableAttributeList)
print(RelationList)
print(EnumList)
print(EnumValueList)

[['jam', 'OtherIncome'], ['jam', 'Annuity']]
[['jam', 'OtherIncome', 'otherIncomeID', 'guid', '', True, 'Concert internal ID'], ['jam', 'OtherIncome', 'contactID', 'guid', '', False, 'Implicit FK in JSON structure, not required in payload'], ['jam', 'OtherIncome', 'otherIncomeSource', 'enum', 'otherIncomeSourceValues', False, ''], ['jam', 'OtherIncome', 'annualGrossIncome', 'Currency', '', False, ''], ['jam', 'OtherIncome', 'netMonthlyIncome', 'Currency', '', False, ''], ['jam', 'Annuity', 'annuityID', 'guid', '', True, 'Concert internal ID'], ['jam', 'Annuity', 'contactID', 'guid', '', False, 'Implicit FK in JSON structure, not required in payload'], ['jam', 'Annuity', 'name', 'String', '', False, ''], ['jam', 'Annuity', 'pensionType', 'enum', 'pensionIncomeTypeValues', False, ''], ['jam', 'Annuity', 'annuityIncome', 'Currency', '', False, ''], ['jam', 'Annuity', 'lifeCover', 'Boolean', '', False, ''], ['jam', 'Annuity', 'sumAssured', 'Currency', '', False, ''], ['jam', 'Annuity', 'po

In [5]:
# process frame: TableList

df = pd.DataFrame(TableList, columns=['ModelKeyPhrase', 'LogicalEntityName'])
df['RecordSource'] = df.apply(lambda row: RecordSource, axis=1)
df['isDeleted'] = df.apply(lambda row: 0, axis=1)
df['LogicalEntityKeyPhrase'] = df.apply(lambda row: row['ModelKeyPhrase'] + '.' + row['LogicalEntityName'], axis=1)
df['ModelLogicalEntityKeyPhrase'] = df.apply(lambda row: row['ModelKeyPhrase'] + ':' + row['LogicalEntityKeyPhrase'], axis=1)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelHashKey'
  , columns = ['ModelKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'LogicalEntityHashKey'
  , columns = ['LogicalEntityKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = ['LogicalEntityName', 'isDeleted']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelLogicalEntityHashKey'
  , columns = ['ModelLogicalEntityKeyPhrase']
)

df

# Explicit removal is going to be a problem
# No way to say 'this was removed'
# Can't just wipe/ readd because the import might only be a portion of the model
# Defer to excel import. If it appears here, it's not deleted - otherwise we don't know.


,ModelKeyPhrase,LogicalEntityName,RecordSource,isDeleted,LogicalEntityKeyPhrase,ModelLogicalEntityKeyPhrase,ModelHashKey,LogicalEntityHashKey,HashDiff,ModelLogicalEntityHashKey
0,jam,OtherIncome,dbdiagramio,0,jam.OtherIncome,jam:jam.OtherIncome,5275cb415e5bc3948e8f2cd492859f26,aa717daa62d4834ae051be75a8fa9e2a,d037c70be63207730673f03aa71fc900,35d1e1bbabd6060b63d55896fce9e15d
1,jam,Annuity,dbdiagramio,0,jam.Annuity,jam:jam.Annuity,5275cb415e5bc3948e8f2cd492859f26,6c257696c4bc78396a786e55d7543750,35e599c0cf936659310e855aee72778c,0ef4212d74a6c07ee1c95f4ba043ea8d


In [6]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_DbdiagramioToLogical")
conn.commit()
df.to_sql('stg_Py_DbdiagramioToLogical', conn, if_exists='append', index=False)

2

In [7]:
# Write to the Hub (LogicalEntity)
sql_query = """
INSERT INTO rv_h_LogicalEntity
(
    LogicalEntityHashKey
  , LoadDate
  , RecordSource
  , LogicalEntityKeyPhrase
)
SELECT DISTINCT
    LogicalEntityHashKey
  , LoadDate
  , RecordSource
  , LogicalEntityKeyPhrase
FROM
  stg_Py_DbdiagramioToLogical
WHERE
  LogicalEntityHashKey NOT IN (SELECT LogicalEntityHashKey FROM rv_h_LogicalEntity)
""";
conn.execute(sql_query)
conn.commit()

In [8]:
# Write to the Satellite (LogicalEntity)
sql_query = """
INSERT INTO rv_s_LogicalEntity
(
    LogicalEntityHashKey
  , LoadDate
  , RecordSource
  , HashDiff
  , LogicalEntityName
  , isDeleted
)
SELECT DISTINCT
    stg.LogicalEntityHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  , stg.LogicalEntityName
  , stg.isDeleted
FROM
  stg_Py_DbdiagramioToLogical AS stg
  LEFT OUTER JOIN rv_s_LogicalEntity AS sat ON (
    stg.LogicalEntityHashKey = sat.LogicalEntityHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_LogicalEntity AS z
      WHERE z.LogicalEntityHashKey = sat.LogicalEntityHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()

In [9]:
# Write to the Link (Model - LogicalEntity)
sql_query = """
INSERT INTO rv_l_ModelLogicalEntity
(
  ModelLogicalEntityHashKey
  , LoadDate
  , RecordSource
  , ModelHashKey
  , LogicalEntityHashKey
)
SELECT
  stg.ModelLogicalEntityHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.ModelHashKey
  , stg.LogicalEntityHashKey
FROM
  stg_Py_DbdiagramioToLogical stg
WHERE
  stg.ModelLogicalEntityHashKey IS NOT NULL
  AND ModelLogicalEntityHashKey NOT IN (
    SELECT ModelLogicalEntityHashKey
    FROM rv_l_ModelLogicalEntity
  )
""";
conn.execute(sql_query)
conn.commit()

In [10]:
# process frame: TableAttributeList

df = pd.DataFrame(TableAttributeList, columns=['ModelKeyPhrase', 'LogicalEntityName', 'LogicalEntityAttributeName', 'DataType', 'EnumName', 'IsPK', 'Notes'])
df['RecordSource'] = df.apply(lambda row: RecordSource, axis=1)
df['isDeleted'] = df.apply(lambda row: 0, axis=1)
# entity names
df['LogicalEntityKeyPhrase'] = df.apply(lambda row: row['ModelKeyPhrase'] + '.' + row['LogicalEntityName'], axis=1)
df['LogicalEntityAttributeKeyPhrase'] = df.apply(lambda row: row['ModelKeyPhrase'] + '.' + row['LogicalEntityName'] + '.' + row['LogicalEntityAttributeName'], axis=1)
df['EnumKeyPhrase'] = df.apply(lambda row: row['ModelKeyPhrase'] + '.' + row['EnumName'], axis=1)
# link names
df['LogicalEntityLogicalEntityAttributeKeyPhrase'] = df.apply(lambda row: row['LogicalEntityKeyPhrase'] + ':' + row['LogicalEntityAttributeKeyPhrase'], axis=1)
df['LogicalEntityAttributeEnumKeyPhrase'] = df.apply(lambda row: row['LogicalEntityAttributeKeyPhrase'] + ':' + row['EnumKeyPhrase'], axis=1)

# hashing
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'LogicalEntityHashKey'
  , columns = ['LogicalEntityKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'LogicalEntityAttributeHashKey'
  , columns = ['LogicalEntityAttributeKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'EnumHashKey'
  , columns = ['EnumKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = ['LogicalEntityAttributeName', 'isDeleted', 'DataType', 'EnumName', 'IsPK', 'Notes']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'LogicalEntityLogicalEntityAttributeHashKey'
  , columns = ['LogicalEntityLogicalEntityAttributeKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'LogicalEntityAttributeEnumHashKey'
  , columns = ['LogicalEntityAttributeEnumKeyPhrase']
)

df

,ModelKeyPhrase,LogicalEntityName,LogicalEntityAttributeName,DataType,EnumName,IsPK,Notes,RecordSource,isDeleted,LogicalEntityKeyPhrase,LogicalEntityAttributeKeyPhrase,EnumKeyPhrase,LogicalEntityLogicalEntityAttributeKeyPhrase,LogicalEntityAttributeEnumKeyPhrase,LogicalEntityHashKey,LogicalEntityAttributeHashKey,EnumHashKey,HashDiff,LogicalEntityLogicalEntityAttributeHashKey,LogicalEntityAttributeEnumHashKey
0,jam,OtherIncome,otherIncomeID,guid,,True,Concert internal ID,dbdiagramio,0,jam.OtherIncome,jam.OtherIncome.otherIncomeID,jam.,jam.OtherIncome:jam.OtherIncome.otherIncomeID,jam.OtherIncome.otherIncomeID:jam.,aa717daa62d4834ae051be75a8fa9e2a,2ac57aa45a68758d3273529d7bae93cf,31913f7750d4793e2eeca5554211128a,c84e4883d33100dac50e7a1bd630a69d,f650ef31f27db4983b0f04ffc4603dd0,cdac9fa1329a7ea58fbb3b48fa0834cb
1,jam,OtherIncome,contactID,guid,,False,"Implicit FK in JSON structure, not required in...",dbdiagramio,0,jam.OtherIncome,jam.OtherIncome.contactID,jam.,jam.OtherIncome:jam.OtherIncome.contactID,jam.OtherIncome.contactID:jam.,aa717daa62d4834ae051be75a8fa9e2a,995f8ee2cfb0e4375d91e94c30fd7306,31913f7750d4793e2eeca5554211128a,9bfca0442034b111e6e78a4aa6da6e10,faeb9dd3c8ad1f9cf501ea8f7b419c32,d5fe35fce4af5858a3fafac519867ece
2,jam,OtherIncome,otherIncomeSource,enum,otherIncomeSourceValues,False,,dbdiagramio,0,jam.OtherIncome,jam.OtherIncome.otherIncomeSource,jam.otherIncomeSourceValues,jam.OtherIncome:jam.OtherIncome.otherIncomeSource,jam.OtherIncome.otherIncomeSource:jam.otherInc...,aa717daa62d4834ae051be75a8fa9e2a,242743cf6105d9998603bf5a1fe2239e,a5a80c3828e2479a201b178dfdbee58b,85bd713ecb786874a08c1d0ca8461008,a118cf6106cf0ff6d5af7c887f150c56,abc5f99f9bef1b484ed100718a118949
3,jam,OtherIncome,annualGrossIncome,Currency,,False,,dbdiagramio,0,jam.OtherIncome,jam.OtherIncome.annualGrossIncome,jam.,jam.OtherIncome:jam.OtherIncome.annualGrossIncome,jam.OtherIncome.annualGrossIncome:jam.,aa717daa62d4834ae051be75a8fa9e2a,ad892498354296d3881a0b4573d8961c,31913f7750d4793e2eeca5554211128a,16b9a48566adaf865ba6bffbab8d86d3,e7376141781a5b2327ad867ed3567b15,5b451eb91a2e254cf3c274f8b1c32b3b
4,jam,OtherIncome,netMonthlyIncome,Currency,,False,,dbdiagramio,0,jam.OtherIncome,jam.OtherIncome.netMonthlyIncome,jam.,jam.OtherIncome:jam.OtherIncome.netMonthlyIncome,jam.OtherIncome.netMonthlyIncome:jam.,aa717daa62d4834ae051be75a8fa9e2a,cba6c239c7f83ec77a6158525dd14538,31913f7750d4793e2eeca5554211128a,d1e08370260deb74bc78e19618010823,4cd2e8a1bed14bc45ccb56b0bfc5cf16,05d9dd89c1cbee8e9461b9dc8d91a4f9
5,jam,Annuity,annuityID,guid,,True,Concert internal ID,dbdiagramio,0,jam.Annuity,jam.Annuity.annuityID,jam.,jam.Annuity:jam.Annuity.annuityID,jam.Annuity.annuityID:jam.,6c257696c4bc78396a786e55d7543750,d70a496aacaef1037f670ed4b7a73ec5,31913f7750d4793e2eeca5554211128a,c37728ec8a3007dcee309044e763c24a,0d24f76060ba84f8d8d5acea3a01e9a7,c575ce147d3809472c33539d9a9c1376
6,jam,Annuity,contactID,guid,,False,"Implicit FK in JSON structure, not required in...",dbdiagramio,0,jam.Annuity,jam.Annuity.contactID,jam.,jam.Annuity:jam.Annuity.contactID,jam.Annuity.contactID:jam.,6c257696c4bc78396a786e55d7543750,bd9366f165f852bff89e6bb7415b607a,31913f7750d4793e2eeca5554211128a,9bfca0442034b111e6e78a4aa6da6e10,9a201888cb7840d9ce0e1f47c8a0d525,c0585c9ac3b25c9c90588c3a18351c2a
7,jam,Annuity,name,String,,False,,dbdiagramio,0,jam.Annuity,jam.Annuity.name,jam.,jam.Annuity:jam.Annuity.name,jam.Annuity.name:jam.,6c257696c4bc78396a786e55d7543750,bcf1514ac9f3ff431eda027d59b859ba,31913f7750d4793e2eeca5554211128a,65030a909286645d36c8771ab96aeaf9,fd08307aca08d1315a217216b91c31d9,2c8a77915d16fe443327a4ca09e584f3
8,jam,Annuity,pensionType,enum,pensionIncomeTypeValues,False,,dbdiagramio,0,jam.Annuity,jam.Annuity.pensionType,jam.pensionIncomeTypeValues,jam.Annuity:jam.Annuity.pensionType,jam.Annuity.pensionType:jam.pensionIncomeTypeV...,6c257696c4bc78396a786e55d7543750,68636cc109d4f1b5eac8dc6ea2b64c6a,56a03f5f5acc0bee1b96897d54ab1c29,a43744d4ab7c531fea568bb6ccdd7c90,e65da14c0722e620b35276b80

In [11]:
# do an effectivity update on all attributes that have two current linked enums.